# Stage 1: Build a database

## Basic match dataframe setup

In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("football_database.sqlite")
df = pd.read_sql_query("""
        SELECT *
        FROM match LIMIT 5;
        """, conn)

df

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


In [2]:
# Add match_result

df.loc[df['home_team_goal'] > df['away_team_goal'], 'match_result'] = 1
df.loc[df['home_team_goal'] < df['away_team_goal'], 'match_result'] = 2
df.loc[df['home_team_goal'] == df['away_team_goal'], 'match_result'] = 3
df['match_result'] = df['match_result'].astype(int)

# Get the first year of season
df['season'] = df['season'].apply(lambda x: x[:4])

df

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA,match_result
0,1,1,1,2008,1,2008-08-17 00:00:00,492473,9987,9993,1,...,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20,3
1,2,1,1,2008,1,2008-08-16 00:00:00,492474,10000,9994,0,...,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60,3
2,3,1,1,2008,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75,2
3,4,1,1,2008,1,2008-08-17 00:00:00,492476,9991,9998,5,...,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50,1
4,5,1,1,2008,1,2008-08-16 00:00:00,492477,7947,9985,1,...,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67,2


In [3]:
# Calculate lineup average age

# Get average ages of playes in a team
player_bday = pd.read_sql_query("""
        SELECT id, strftime("%Y", birthday) as year
        FROM player
        LIMIT 5
""", conn, params=())

df['home_player_3']

0    None
1    None
2    None
3    None
4    None
Name: home_player_3, dtype: object

In [4]:
# Get match data from XML
